In [1]:
import pandas as pd
import xlrd
import psycopg2
from sqlalchemy import create_engine
import boto3
import botocore
import requests
import os
from io import StringIO

## CLEANING AND UPLOADING FILES TO NEW BUCKET

In [22]:
# Define the S3 bucket URLs
source_bucket_url = 'https://inep.s3.amazonaws.com/'
destination_bucket_url = 'https://inep-cleaned.s3.amazonaws.com/'

# List of years for your files
years = ['2020', '2021', '2022']

for year in years:
    # Step 1: Download the file
    print(f"Step 1: Downloading {year} file...")
    source_key = f'MICRODADOS_ENEM_{year}.csv'
    destination_key = f'summary_{source_key}'
    
    source_object_url = source_bucket_url + source_key
    response = requests.get(source_object_url)
    
    if response.status_code == 200:
        with open(source_key, 'wb') as local_file:
            local_file.write(response.content)
    else:
        print(f"Failed to download {year} file. Status code: {response.status_code}")
    
        """# Print the response content for more details if available
        if response.content:
            print("Response Content:")
            print(response.content.decode('utf-8'))

        # Print headers for additional information if available
        if response.headers:
            print("Response Headers:")
            for header, value in response.headers.items():
                print(f"{header}: {value}")"""

    # Step 2: Create a DataFrame and select specific columns
    print(f"Step 2: Creating DataFrame for {year} file...")
    columns_to_keep = [
        'NU_INSCRICAO', 'NU_ANO', 'TP_FAIXA_ETARIA', 'TP_SEXO', 'TP_ESTADO_CIVIL', 'TP_COR_RACA',
        'TP_NACIONALIDADE', 'TP_ST_CONCLUSAO', 'TP_ANO_CONCLUIU', 'TP_ESCOLA', 'TP_ENSINO',
        'IN_TREINEIRO', 'CO_MUNICIPIO_PROVA', 'NO_MUNICIPIO_PROVA', 'CO_UF_PROVA', 'SG_UF_PROVA',
        'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT', 'NU_NOTA_REDACAO'
    ]
    
    df_MICRODADOS_ENEM_{year} = pd.read_csv(source_key, sep=';', encoding='latin1', usecols=columns_to_keep)
    
    # Step 3: Rename columns
    print(f"Step 3: Renaming columns for {year} file...")
    df_MICRODADOS_ENEM_{year}.rename(columns={
        'NU_INSCRICAO': 'id', 'NU_ANO': 'year', 'TP_FAIXA_ETARIA': 'age_code', 'TP_SEXO': 'sex_code',
        'TP_ESTADO_CIVIL': 'civil_code', 'TP_COR_RACA': 'etinicity_code', 'TP_NACIONALIDADE': 'nationality_code',
        'TP_ST_CONCLUSAO': 'conclusion_code', 'TP_ANO_CONCLUIU': 'concluion_year', 'TP_ESCOLA': 'school_code',
        'TP_ENSINO': 'teaching_code', 'IN_TREINEIRO': 'is_training', 'CO_MUNICIPIO_PROVA': 'municipality_code',
        'NO_MUNICIPIO_PROVA': 'municipality_name', 'CO_UF_PROVA': 'uf_code', 'SG_UF_PROVA': 'uf_name',
        'NU_NOTA_CN': 'cn_score', 'NU_NOTA_CH': 'ch_score', 'NU_NOTA_LC': 'lc_score',
        'NU_NOTA_MT': 'mt_score', 'NU_NOTA_REDACAO': 'essay_score'
    }, inplace=True)
    
    # Step 4: Export the cleaned data to a new CSV
    print(f"Step 4: Exporting cleaned data for {year}...")
    df_MICRODADOS_ENEM_{year}.to_csv(destination_key, index=False)
    
    # Step 5: Upload the CSV to the destination bucket
    print(f"Step 5: Uploading cleaned data to {destination_bucket_url}...")
    destination_object_url = destination_bucket_url + destination_key
    with open(destination_key, 'rb') as local_file:
        response = requests.put(destination_object_url, data=local_file)
    
    if response.status_code == 200:
        print(f"Cleaning and uploading for {year} is complete!\n")
    else:
        print(f"Failed to upload {year} file. Status code: {response.status_code}")


Step 1: Downloading 2020 file...
Step 2: Creating DataFrame for 2020 file...
Step 3: Renaming columns for 2020 file...
Step 4: Exporting cleaned data for 2020...
Step 5: Uploading cleaned data to https://inep-cleaned.s3.amazonaws.com/...
Cleaning and uploading for 2020 is complete!

Step 1: Downloading 2021 file...
Step 2: Creating DataFrame for 2021 file...
Step 3: Renaming columns for 2021 file...
Step 4: Exporting cleaned data for 2021...
Step 5: Uploading cleaned data to https://inep-cleaned.s3.amazonaws.com/...
Cleaning and uploading for 2021 is complete!

Step 1: Downloading 2022 file...
Step 2: Creating DataFrame for 2022 file...
Step 3: Renaming columns for 2022 file...
Step 4: Exporting cleaned data for 2022...
Step 5: Uploading cleaned data to https://inep-cleaned.s3.amazonaws.com/...
Cleaning and uploading for 2022 is complete!



In [24]:
"""# Cleanup: Delete the downloaded files
os.remove(source_key)
os.remove(destination_key)"""

## UPLOADING DATA TO POSTGRES

In [2]:
# Specify the S3 bucket name and file names
bucket_name = 'inep-cleaned'
file_names = [
    'summary_MICRODADOS_ENEM_2020.csv',
    'summary_MICRODADOS_ENEM_2021.csv',
    'summary_MICRODADOS_ENEM_2022.csv'
]

# Initialize an empty list to store individual DataFrames
dataframes = []

# Iterate through the specified file names and download each one
for file_name in file_names:
    # Construct the public S3 URL
    s3_url = f'https://{bucket_name}.s3.amazonaws.com/{file_name}'
    
    # Download the CSV file using requests
    response = requests.get(s3_url)
    
    if response.status_code == 200:
        # Read the CSV data into a pandas DataFrame
        df = pd.read_csv(StringIO(response.text))
        dataframes.append(df)
    else:
        print(f"Failed to download {file_name}")

# Combine all DataFrames into a single DataFrame
combined_df = pd.concat(dataframes, ignore_index=True)
combined_df.head(10)

,id,year,age_code,sex_code,civil_code,etinicity_code,nationality_code,conclusion_code,concluion_year,school_code,...,is_training,municipality_code,municipality_name,uf_code,uf_name,cn_score,ch_score,lc_score,mt_score,essay_score
0,200006271946,2020,11,F,1,2,1,1,11,1,...,0,1501402,Belém,15,PA,NaN,NaN,NaN,NaN,NaN
1,200001195856,2020,11,M,2,3,1,1,11,1,...,0,2408102,Natal,24,RN,604.1,661.7,595.3,711.3,580.0
2,200001943954,2020,4,F,2,3,2,2,0,2,...,0,2927408,Salvador,29,BA,NaN,NaN,NaN,NaN,NaN
3,200001908998,2020,2,M,1,3,1,2,0,2,...,0,3547304,Santana de Parnaíba,35,SP,620.8,675.0,624.2,759.4,760.0
4,200001634757,2020,4,F,1,3,2,1,1,1,...,0,3121605,Diamantina,31,MG,NaN,NaN,NaN,NaN,NaN
5,200003132410,2020,3,F,1,3,1,1,1,1,...,0,4305207,Cerro Largo,43,RS,498.1,604.7,505.4,526.7,700.0
6,200001379770,2020,9,M,1,3,1,1,6,1,...,0,2611606,Recife,26,PE,NaN,NaN,NaN,NaN,NaN
7,200001334237,2020,4,M,1,1,1,1,1,1,...,0,3550308,São Paulo,35,SP,604.6,604.8,562.1,753.2,600.0
8,200006762554,2020,5,F,2,3,1,1,3,1,...,0,2507507,João Pessoa,25,PB,439.7,383.5,486.2,448.5,600.0
9,200005146210,2020,2,M,1,2,1,2,0,2,...,0,2304400,Fortaleza,23,CE,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Set up a connection to your PostgreSQL RDS instance
host = 'database-2.ckuaogoaistw.us-east-1.rds.amazonaws.com'
port = 5432
database = 'postgres'
user = 'postgres'
password = 'postgres'

connection = psycopg2.connect(
    host=host,
    port=port,
    database=database,
    user=user,
    password=password
)

# create a cursor object
cursor = connection.cursor()

# create table
table_name = 'inep_data' 

# Set up a SQLAlchemy engine
engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{database}')

# Use Pandas to create the table and insert data
combined_df.to_sql(
    table_name,
    engine,
    if_exists='replace',  # You can use 'replace', 'append', or 'fail' depending on your needs
    index=False  # Set to False if you don't want to include the DataFrame index as a column
)

# Commit changes and close the cursor and connection
connection.commit()
cursor.close()
connection.close()